# Registering Layers to RW-API

We add the following [Aqueduct floods assets](https://code.earthengine.google.com/4ee58c4281e66ef627704996adb28073) as layers into the RW API

#### Useful links
- [RW API documentation](https://resource-watch.github.io/doc-api)
- [RW example notebooks repo](https://github.com/Vizzuality/data_sci_tutorials/blob/master/work/RW_API_CARTO_to_layers.ipynb)
- All RW API datasets: http://api.resourcewatch.org/v1/dataset?page[number]=1&page[size]=1000
- All RW API layers: http://api.resourcewatch.org/v1/layer?page[number]=1&page[size]=1000

In [1]:
import json
import requests
from pprint import pprint
import random
import numpy as np
import time
import getpass
import ee

In [2]:
s = requests.session()
s.cookies.clear()

In [3]:
prod_server = "https://api.resourcewatch.org"
def auth():
    with requests.Session() as s:
        headers = {'Content-Type': 'application/json'}
        payload = json.dumps({ 'email': f'{input("Email: ")}',
                               'password': f'{getpass.getpass(prompt="Password: ")}'})
        response = s.post(f'{prod_server}/auth/login',  headers = headers,  data = payload)
        response.raise_for_status()
    return response.json().get('data').get('token')

OAUTH = auth()

Email: alicia.arenzana@vizzuality.com
Password: ········


***
## Register Dataset

The datasets must be registered first, then the layer attributed to it later.

The only fields that will need to be modified for each layer are: `connectorUrl`, and `name`.


### Defaults

By default, the `application` should be an array with 'aqueduct': `["aqueduct"]`

For Earth Engine layers, the `provider` field should always be set as `gee` and the `connectorType` set as `rest` (Note: this is different for different dataset types)

### Other Fields

In the case of Earth Engine layers, the `tableName` field should contain the asset url, e.g. `projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation` 

In [ ]:
# EXAMPLE payload
# Note: Get connector table from layerspec --> 'table_name'.

payload = {"dataset": {
  "connectorType": "rest", 
  "provider":"gee", 
  "type": "raster",
  "application": ["aqueduct"],
  "tableName": "projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation",
  "name": "Aqueduct_floods"
    }
  }

In [ ]:
#Post new dataset

url = f'{prod_server}/v1/dataset'

headers = {'Authorization': f'Bearer {OAUTH}', 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.json())

## Updating Datasets

In [4]:
""" Copy the old one and change desired values (in this case attributes.description)
    NOTE that you cannot update id, or type, only attributes!
"""

payload = {
    "type": "raster"
}

In [5]:
#Update datasets
dataset_id = 'cbe2c493-125a-41b3-a401-98deb000d57e'

url = f'{prod_server}/v1/dataset/{dataset_id}'

headers = {'Authorization': f'Bearer {OAUTH}', 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}

r = requests.patch(url, data=json.dumps(payload), headers=headers)

pprint(r.json())

{'data': {'attributes': {'application': ['aqueduct'],
                         'attributesPath': None,
                         'clonedHost': {},
                         'connectorType': 'rest',
                         'connectorUrl': None,
                         'createdAt': '2021-02-24T09:17:20.916Z',
                         'dataLastUpdated': None,
                         'dataPath': None,
                         'env': 'production',
                         'errorMessage': '',
                         'geoInfo': False,
                         'layerRelevantProps': [],
                         'legend': {'binary': [],
                                    'boolean': [],
                                    'byte': [],
                                    'country': [],
                                    'date': [],
                                    'double': [],
                                    'float': [],
                                    'half_float': [],
            

## Deleting Datasets

In [ ]:
#Delete dataset
dataset_id = "aba995e4-7171-4e5a-9356-05041e6b2d4a"

url = f'http://api.resourcewatch.org/dataset/{dataset_id}'

headers = {'Authorization': 'Bearer ' + OAUTH, 'Content-Type': 'application/json', 'Cache-Control': 'no-cache'}

r = requests.delete(url, headers=headers)
print(r.url)

pprint(r.json())

***
## Register Vocabulary

In [4]:
vocabulary_name = 'aqueductfloods_v2'

payload = {
    "application":'aqueduct',
   "name": vocabulary_name
  }

In [5]:
#Register vocabulary

url = f'https://api.resourcewatch.org/v1/vocabulary/'

headers = {'Authorization': f'Bearer {OAUTH}', 'Content-Type': 'application/json'}

r = requests.post(url, data=json.dumps(payload), headers=headers)
print(r.url)
pprint(r.json())
layer = r.json().get('data', None)

https://api.resourcewatch.org/v1/vocabulary/
{'data': [{'attributes': {'application': 'aqueduct',
                          'name': 'aqueductfloods_v2',
                          'resources': []},
           'id': 'aqueductfloods_v2',
           'type': 'vocabulary'}]}


***

## Registering Layers

Now we have a dataset added we can associate a new layer with it. Each layer is comprised of 4 parts: 

1. the basic information
    - name, description, provider, iso, published
2. layer info
    - layerConfig (how the layer will retieve data and how it will look)   
3. legend info
    - type of legend  
4. interactivity
    - how the tool-tips behave

In [7]:
earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=NlzMQC8F5-BGMJjPf8T2Ijj2MYZBniHymLFTf7y_LVI&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: ^C
Traceback (most recent call last):
  File "/opt/conda/bin/earthengine", line 8, in <module>
    sys.exit(main())
  File "/opt/conda/lib/python3.8/site-packages/ee/cli/eecli.py", line 84, in main
    _run_command()
  File "/opt/conda/lib/python3.8/site-packages/ee/cli/ee

**Get the features' (layers) information of the Image Collection**

In [9]:
ee.Initialize()

In [10]:
myImageCollection = ee.ImageCollection('projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation')

In [11]:
print(myImageCollection)

ee.ImageCollection({
  "functionInvocationValue": {
    "functionName": "ImageCollection.load",
    "arguments": {
      "id": {
        "constantValue": "projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation"
      }
    }
  }
})


In [12]:
features = myImageCollection.getInfo()['features']

In [13]:
len(features)

689

In [14]:
features[123]

{'type': 'Image',
 'bands': [{'id': 'b1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [43200, 21600],
   'crs': 'EPSG:4326',
   'crs_transform': [0.008333333333333333,
    0,
    -180,
    0,
    -0.008333333333333333,
    90]}],
 'id': 'projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation/inuncoast_rcp4p5_nosub_2080_rp0025_0_perc_05',
 'version': 1561638105263820.0,
 'properties': {'floodtype': 'inuncoast',
  'Conventions': 'CF-1.6',
  'references': 'http://floods.wri.org/',
  'year': 2080,
  'inun_standard_name': 'water_surface_height_above_reference_datum',
  'project': 'Aqueduct Global Flood Analyzer',
  'returnperiod': 'rp0025',
  'ingested_by': 'rutgerhofste',
  'source': 'Global Tide and Surge Model',
  'title': 'Aqueduct Coastal hazard layer',
  'sea_level_rise_scenario': 5,
  'institution': 'Deltares',
  'inun_comment': 'water_surface_reference_datum_altitude is given in file /p/1209884-aqueduct/Datasets/M

In [15]:
features[0].get('id')

'projects/WRI-Aquaduct/floods/Y2018M08D16_RH_Floods_Inundation_EE_V01/output_V06/inundation/inuncoast_historical_nosub_hist_rp0001_5'

In [16]:
features[0].get('id').split("/",6)[6]

'inuncoast_historical_nosub_hist_rp0001_5'

In [17]:
features[0].get('properties').get('returnperiod')

'rp0001'

In [18]:
all_tags = ['floodtype', 'year', 'climate', 'model', 'subsidence', 'sea_level_rise_scenario']

In [19]:
for j in np.linspace(0,688,689):
    tags = []
    s = ''
    for n, i in enumerate(all_tags):
        if n != 0: 
            s+=str('_')
        #if i == 'year' and features[int(j)].get('properties').get(i) != None:
        #    s+='{:04.0f}'.format(features[int(j)].get('properties').get(i))
        #else:
        #    s+=str(features[int(j)].get('properties').get(i))
        s+=str(features[int(j)].get('properties').get(i))
    tags.append(s)
    tags.append(features[int(j)].get('properties').get('returnperiod'))
    print(tags)

['inuncoast_None_historical_None_nosub_None', 'rp0001']
['inuncoast_None_historical_None_nosub_None', 'rp0002']
['inuncoast_None_historical_None_nosub_None', 'rp0005']
['inuncoast_None_historical_None_nosub_None', 'rp0010']
['inuncoast_None_historical_None_nosub_None', 'rp0025']
['inuncoast_None_historical_None_nosub_None', 'rp0050']
['inuncoast_None_historical_None_nosub_None', 'rp0100']
['inuncoast_None_historical_None_nosub_None', 'rp0250']
['inuncoast_None_historical_None_nosub_None', 'rp0500']
['inuncoast_None_historical_None_nosub_None', 'rp1000']
['inuncoast_2030_historical_None_wtsub_None', 'rp0001']
['inuncoast_2030_historical_None_wtsub_None', 'rp0002']
['inuncoast_2030_historical_None_wtsub_None', 'rp0005']
['inuncoast_2030_historical_None_wtsub_None', 'rp0010']
['inuncoast_2030_historical_None_wtsub_None', 'rp0025']
['inuncoast_2030_historical_None_wtsub_None', 'rp0050']
['inuncoast_2030_historical_None_wtsub_None', 'rp0100']
['inuncoast_2030_historical_None_wtsub_None', 'r

**Unique values**

In [20]:
floodtype = []
returnperiod = []
year = []
climate = []
model = []
subsidence = []
sea_level_rise_scenario = []

for i in range(len(features)):
    floodtype.append(features[i].get('properties').get('floodtype'))
    returnperiod.append(features[i].get('properties').get('returnperiod'))
    year.append(features[i].get('properties').get('year'))
    climate.append(features[i].get('properties').get('climate'))
    model.append(features[i].get('properties').get('model'))
    subsidence.append(features[i].get('properties').get('subsidence'))
    sea_level_rise_scenario.append(features[i].get('properties').get('sea_level_rise_scenario'))

In [21]:
set(floodtype)

{'inuncoast', 'inunriver'}

In [22]:
set(returnperiod)

{'rp00002',
 'rp00005',
 'rp0001',
 'rp00010',
 'rp0002',
 'rp00025',
 'rp0005',
 'rp00050',
 'rp0010',
 'rp00100',
 'rp0025',
 'rp00250',
 'rp0050',
 'rp00500',
 'rp0100',
 'rp01000',
 'rp0250',
 'rp0500',
 'rp1000'}

In [23]:
set(year)

{1980, 2030, 2050, 2080, None}

In [24]:
set(climate)

{'historical', 'rcp4p5', 'rcp8p5'}

In [25]:
set(model)

{'000000000WATCH',
 '00000NorESM1-M',
 '0000GFDL-ESM2M',
 '0000HadGEM2-ES',
 '00IPSL-CM5A-LR',
 'MIROC-ESM-CHEM',
 None,
 'perc'}

In [26]:
set(subsidence)

{None, 'nosub', 'wtsub'}

In [27]:
set(sea_level_rise_scenario)

{5, 50, None}

**Save each feature as a layer in the RW API**

In [28]:
change_returnperiod = {"rp00001": "rp0001", "rp00002": "rp0002", "rp00005": "rp0005",
                      "rp00010": "rp0010", "rp00025": "rp0025", "rp00050": "rp0050", 
                      "rp00100": "rp0100", "rp00250": "rp0250", "rp00500": "rp0500", "rp01000": "rp1000"}

color_dict = {
    "rp1000": "#91FEE3",
    "rp0500": "#88F6DB",
    "rp0250": "#6BD9BF",
    "rp0100": "#00B4BA",
    "rp0050": "#008EB3",
    "rp0025": "#0066A4",
    "rp0010": "#003E88",
    "rp0005": "#0A125E",
    "rp0002": "#070e49"
}

all_tags = ['floodtype', 'year', 'climate', 'model', 'subsidence', 'sea_level_rise_scenario']

In [29]:
dataset_id = 'cbe2c493-125a-41b3-a401-98deb000d57e'

#for i in np.arange(403,len(features)):
for i in range(len(features)):
    time.sleep(5)
    print(i)
    returnperiod = features[i].get('properties').get('returnperiod')

    # Remove returnperiod name inconsistency
    if returnperiod in change_returnperiod.keys():
        returnperiod = change_returnperiod[returnperiod]

    if returnperiod != "rp0001":
        # Add color to each `returnperiod`
        color = color_dict[returnperiod]
    
        payload = {
        "application": ["aqueduct"],
         "name": features[i].get('id').split("/",6)[6],
         "provider": "gee",
         "layerConfig": {
                "assetId": features[i].get('id'),
                "type": "gee",
                "body": {
                    "sldValue": f"<RasterSymbolizer> \
              <Opacity>1.0</Opacity> \
                <ChannelSelection> \
                  <GrayChannel> \
                      <SourceChannelName>1</SourceChannelName> \
                  </GrayChannel> \
              </ChannelSelection> \
              <ColorMap type=\"ramp\" extended=\"false\" > \
                <ColorMapEntry color=\"{color}\" quantity=\"0\" opacity=\"0\" /> \
                <ColorMapEntry color=\"{color}\" quantity=\"1\" opacity=\"1\" /> \
                </ColorMap> \
             </RasterSymbolizer>",
                "styleType": "sld"
                    }
                },
         "legendConfig": {
            "type": "basic",
            "items": [
              {
                "name": returnperiod,
                "color": color
              }
            ]
          },
          "published": True,
          "default": True
        }    
        
        # Register layers
        url = f'http://api.resourcewatch.org/dataset/{dataset_id}/layer/'

        headers = {'Authorization': f'Bearer {OAUTH}', 'Content-Type': 'application/json'}

        r = requests.post(url, data=json.dumps(payload), headers=headers)
        #layer = r.json().get('data',None)
        
        # Create a vocabulary
        tags = []
        s = ''
        for n, j in enumerate(all_tags):
            if n != 0: 
                s+=str('_')
            s+=str(features[i].get('properties').get(j))
        tags.append(s)
        tags.append(returnperiod)

        payload_voc = {
                "application": "aqueduct",
                "tags": tags
        }     
        print(payload_voc)
        
        #Register vocabulary
        layer_id = r.json().get('data').get('id')
        print('layer_id:', layer_id)
        
        url = f'http://api.resourcewatch.org/dataset/{dataset_id}/layer/{layer_id}/vocabulary/aqueductfloods_v2'

        headers = {'Authorization': f'Bearer {OAUTH}', 'Content-Type': 'application/json'}

        r = requests.post(url, data=json.dumps(payload_voc), headers=headers)

0
1
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0002']}
layer_id: 43bbbe43-6f19-449b-adac-cc66a1b7ae54
2
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0005']}
layer_id: d662ac01-afd9-4fab-99bb-4e85ac60c855
3
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0010']}
layer_id: 7301cbac-a8a8-4835-ae6e-ad80cab0e870
4
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0025']}
layer_id: c2f2055f-dc7e-4868-8058-e3c178497625
5
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0050']}
layer_id: c57d48c3-60e0-499e-9023-be07e1c05d9c
6
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0100']}
layer_id: dc37c893-a71a-4c74-be4e-4e527715fedc
7
{'application': 'aqueduct', 'tags': ['inuncoast_None_historical_None_nosub_None', 'rp0250']}
layer_id: 86f9f3ab-ba62-4b0a-acea-a4e359e21b1b
8
{'

66
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_perc_nosub_5', 'rp0050']}
layer_id: bf4cbd26-d787-46ad-869d-a46a82b58d78
67
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_perc_nosub_50', 'rp0050']}
layer_id: 16e1ad87-17fe-463d-bfb8-fca8400e842c
68
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_None_nosub_None', 'rp0100']}
layer_id: 73459c6c-3b0a-42ee-ac35-6aafea22fdfb
69
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_perc_nosub_5', 'rp0100']}
layer_id: daa64f34-11b8-4e8e-9fa3-8af8b9320a13
70
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_perc_nosub_50', 'rp0100']}
layer_id: 9d564109-86f0-4dd4-b0ea-ce19b170c7ec
71
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_None_nosub_None', 'rp0250']}
layer_id: 2fbdc972-551c-480b-9565-fe69e039c106
72
{'application': 'aqueduct', 'tags': ['inuncoast_2030_rcp4p5_perc_nosub_5', 'rp0250']}
layer_id: d965ae1b-7d02-4576-968b-4edee48f7d46
73
{'application': 'aqueduct', 'tags': [

132
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_nosub_5', 'rp0250']}
layer_id: 7d767119-f5dd-4bbd-a189-45c3d4a94d16
133
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_nosub_50', 'rp0250']}
layer_id: d0e89f6e-ada1-42f3-b82f-9af0787cd696
134
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_None_nosub_None', 'rp0500']}
layer_id: c41d9225-8860-4267-a574-a5a825561b83
135
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_nosub_5', 'rp0500']}
layer_id: 1f27e54a-e590-4ea4-a9f0-29fe3f1813fe
136
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_nosub_50', 'rp0500']}
layer_id: 096d581a-8a2e-4bc2-85ec-4cb507746c55
137
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_None_nosub_None', 'rp1000']}
layer_id: 65085c06-8c07-45be-8fb0-ef230a746741
138
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_nosub_5', 'rp1000']}
layer_id: 3f2ac6e9-bcf1-46bb-a3a3-33f1e1e6efea
139
{'application': 'aqueduct', '

198
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp4p5_perc_wtsub_5', 'rp1000']}
layer_id: e5741b31-4435-4238-95d4-6a5e5a213921
199
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp4p5_perc_wtsub_50', 'rp1000']}
layer_id: d5473d74-9aab-41ad-85a6-681c7c633a34
200
201
202
203
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_None_wtsub_None', 'rp0002']}
layer_id: 9b685cbe-c742-49d7-824f-18dc649cd3e9
204
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_wtsub_5', 'rp0002']}
layer_id: 444f60ce-fbaa-458f-87aa-1c666c90f046
205
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_wtsub_50', 'rp0002']}
layer_id: 224897c8-e30f-4b6c-896c-105894f3fd5b
206
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_None_wtsub_None', 'rp0005']}
layer_id: d79ca915-579b-480f-be8c-1f77cf5e968a
207
{'application': 'aqueduct', 'tags': ['inuncoast_2080_rcp4p5_perc_wtsub_5', 'rp0005']}
layer_id: d377a32f-5343-4b87-b664-6ac5e3c6752a
208
{'application': '

266
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_None_nosub_None', 'rp0005']}
layer_id: 2f2939d2-b264-4fdc-83df-8f23de6bcd24
267
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_perc_nosub_5', 'rp0005']}
layer_id: a7f16478-63ff-4bbd-bbb8-f23f622596ca
268
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_perc_nosub_50', 'rp0005']}
layer_id: d355e9d1-d048-47fa-9661-a331abcb37b1
269
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_None_nosub_None', 'rp0010']}
layer_id: 856a6a08-48bb-4160-a0eb-b4382f0094b4
270
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_perc_nosub_5', 'rp0010']}
layer_id: 184897c0-f6c5-42a0-9158-1ed93218fa05
271
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_perc_nosub_50', 'rp0010']}
layer_id: 8df80c2e-97d2-46fb-a33d-f19eac2796c0
272
{'application': 'aqueduct', 'tags': ['inuncoast_2050_rcp8p5_None_nosub_None', 'rp0025']}
layer_id: f719a1fc-17b2-43dc-959b-93a5415382a1
273
{'application': 'aqueduct'

...

454
{'application': 'aqueduct', 'tags': ['inunriver_2030_rcp4p5_0000GFDL-ESM2M_None_None', 'rp1000']}
layer_id: 53c21f0d-e41a-4b93-9b25-a3817e9ecc2a
455
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_0000GFDL-ESM2M_None_None', 'rp0002']}
layer_id: e2214127-345e-46d5-9a09-630581cca9ed
456
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_0000GFDL-ESM2M_None_None', 'rp0005']}
layer_id: d40acc56-262e-4c2a-ae30-ba36e73656e7
457
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_0000GFDL-ESM2M_None_None', 'rp0010']}
layer_id: 39334052-0009-4fe1-af84-755d71bfbb8f
458
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_0000GFDL-ESM2M_None_None', 'rp0025']}
layer_id: 9d2a149a-e9ca-4b70-b19b-0e515b2db38f
459
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_0000GFDL-ESM2M_None_None', 'rp0050']}
layer_id: 3ba98ac2-384e-48e3-bd09-918b2186cdef
460
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_0000GFDL-ESM2M_None_None', 'rp0100']}
laye

509
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0002']}
layer_id: c49b6da1-7ce7-4694-b699-cd3d02bf4e83
510
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0005']}
layer_id: 2aeabf18-56f0-47be-b0b3-b149473ba477
511
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0010']}
layer_id: 00cbb9ea-ce94-4e98-b88a-8aeceb91444c
512
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0025']}
layer_id: a97b5016-ec92-41e1-96fa-49afec7444dc
513
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0050']}
layer_id: f454b5e4-f005-4406-bb15-15ab4ef1836d
514
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0100']}
layer_id: b2224a73-ce1f-484d-a295-b74a4f0b8de3
515
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp4p5_00IPSL-CM5A-LR_None_None', 'rp0250']}
laye

564
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0005']}
layer_id: 4de5c8b8-9391-4a2f-aef0-4d2e070bf68f
565
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0010']}
layer_id: b09b2dfc-763e-4d67-b232-b7502426737e
566
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0025']}
layer_id: b30fe64d-4d99-44d5-a4e2-0eee83ade40f
567
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0050']}
layer_id: 06422a35-bad0-4d19-977f-00d935c133a3
568
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0100']}
layer_id: 52cda162-6639-4ac2-b7fb-de655fd5db23
569
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0250']}
layer_id: 30b12f62-fdf6-48ed-b9d3-c47920855a91
570
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_00000NorESM1-M_None_None', 'rp0500']}
laye

619
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp0010']}
layer_id: a98559c4-d1f6-49ed-b555-84aa9c63140f
620
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp0025']}
layer_id: d37b06c8-f449-4ab8-a54c-2021471f2c9f
621
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp0050']}
layer_id: efcfbf5b-5774-4d5e-9853-cc8f16b2962e
622
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp0100']}
layer_id: 4b7c270c-d2ba-4871-a721-7b726f69d6ab
623
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp0250']}
layer_id: 5c47c00b-fe2c-4685-97da-12fbd7d9b308
624
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp0500']}
layer_id: a2e2d835-05bc-4734-a401-428ee7d09dbe
625
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_0000HadGEM2-ES_None_None', 'rp1000']}
laye

674
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp0025']}
layer_id: 4d5a4ec9-c80b-44df-b9dd-275181213bc3
675
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp0050']}
layer_id: 96ae0ed0-9302-4b68-8d73-9d9b2e7f9a07
676
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp0100']}
layer_id: da1166fd-9b2d-4d6f-a8ac-df2558543a3c
677
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp0250']}
layer_id: 4b686e01-c831-47a8-a4e9-df150d63a3f9
678
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp0500']}
layer_id: 1b90755a-fa09-4c22-9b04-cea21d5eeac9
679
{'application': 'aqueduct', 'tags': ['inunriver_2050_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp1000']}
layer_id: ac55ea0b-6f29-47f8-88e2-886c3c9c1d50
680
{'application': 'aqueduct', 'tags': ['inunriver_2080_rcp8p5_MIROC-ESM-CHEM_None_None', 'rp0002']}
laye

**Get the vocabulary of a given layer**

https://api.resourcewatch.org/v1/dataset/cbe2c493-125a-41b3-a401-98deb000d57e/layer/df874bf8-2073-45db-b9ac-d906dac74af0/vocabulary?application=aqueduct

**Filter by tags**

http://api.resourcewatch.org/v1/dataset/cbe2c493-125a-41b3-a401-98deb000d57e/layer/vocabulary/find?aquaductfloods=inunriver

http://api.resourcewatch.org/v1/dataset/aba995e4-7171-4e5a-9356-05041e6b2d4a/layer/vocabulary/find?aquaductfloods=rp01000&inuriver

**mock-up**

```
eae3b745-ba20-4dd2-8d18-6ea717d852fc ["inunriver","rp00002","1980","historical","000000000WATCH","0"],
c832246e-7c74-4ef3-bf2a-fb13431d3972 ["inunriver","rp00005","2030","rcp4p5","00000NorESM1-M","0"],
2b724dde-1ded-4260-846a-b8619185aa35 ["inunriver","rp00025","2050","rcp4p5","00000NorESM1-M","0"],
4bd8df2d-e6da-412d-8567-7ee14a42ab36 ["inunriver","rp00002","2080","rcp4p5","00000NorESM1-M","0"],


2e7c9ffc-9dec-41d2-8100-546111932a32 ["inuncoast","rp1000","2010","historical","wtsub","0"],
01114dda-cf60-4342-8772-6a83ac427e0c ["inuncoast","rp0010","2030","rcp4p5","wtsub","0"],
844afceb-8bd6-4108-a9aa-c7ceb5a05cee ["inuncoast","rp0005","2050","rcp4p5","perc","wtsub","5"],
08f85122-3c2f-4784-b865-4931f62c2149 ["inuncoast","rp0002","2080","rcp8p5","perc","nosub","50"],
```

Old dataset

In [ ]:
'aba995e4-7171-4e5a-9356-05041e6b2d4a'

Delete all layers from dataset

In [ ]:
url = f'http://api.resourcewatch.org/dataset/{dataset_id}/layer'

payload_d = {
                "application": "aqueduct",
                "page[size]": 10000000
        }

headers = {'Authorization': f'Bearer {OAUTH}', 'Content-Type': 'application/json'}

r = requests.get(url, data=json.dumps(payload_d), headers=headers)